In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
#Initialize Spark Session
spark = SparkSession.builder.appName("TestApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:45:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/22 08:45:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sc=spark.sparkContext

In [7]:
customerDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/customer.csv")
lineitemDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/lineitem.csv")
nationDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/nation.csv")
ordersDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/orders.csv")
partDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/part.csv")
partsuppDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/partsupp.csv")
regionDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/region.csv")
supplierDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/supplier.csv")

In [9]:
customerDf.printSchema()

root
 |-- C_CUSTKEY: integer (nullable = true)
 |-- C_NAME: string (nullable = true)
 |-- C_ADDRESS: string (nullable = true)
 |-- C_NATIONKEY: integer (nullable = true)
 |-- C_PHONE: string (nullable = true)
 |-- C_ACCTBAL: double (nullable = true)
 |-- C_MKTSEGMENT: string (nullable = true)
 |-- C_COMMENT: string (nullable = true)



In [10]:
ordersDf.printSchema()

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)



In [12]:
joinDf = customerDf.join(ordersDf,customerDf.C_CUSTKEY==ordersDf.O_CUSTKEY)
joinDf.show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...|         1|      370|            O|   172799.49| 1996-01-02|          5-LOW|Clerk#000000951|             0|nstructions sl

In [15]:
from pyspark.sql.functions import col
joinDf = joinDf.drop(col("O_CUSTKEY"))
joinDf.show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|O_ORDERKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...|         1|            O|   172799.49| 1996-01-02|          5-LOW|Clerk#000000951|             0|nstructions sleep...|
|      781|Customer#000000781|FQ

In [16]:
ordersDf.show(5)

+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|         1|      370|            O|   172799.49| 1996-01-02|          5-LOW|Clerk#000000951|             0|nstructions sleep...|
|         2|      781|            O|    38426.09| 1996-12-01|       1-URGENT|Clerk#000000880|             0| foxes. pending a...|
|         3|     1234|            F|    205654.3| 1993-10-14|          5-LOW|Clerk#000000955|             0|sly final account...|
|         4|     1369|            O|    56000.91| 1995-10-11|          5-LOW|Clerk#000000124|             0|sits. slyly regul...|
|         5|      445|            F|   105367.67| 1994-07-30|          5-LOW|Clerk#0000009

In [17]:
groupDf = ordersDf.groupBy("O_CLERK").sum("O_TOTALPRICE")

In [18]:
groupDf.show(5)

+---------------+------------------+
|        O_CLERK| sum(O_TOTALPRICE)|
+---------------+------------------+
|Clerk#000000024|        2078084.41|
|Clerk#000000288|        1431028.03|
|Clerk#000000236|        2571896.97|
|Clerk#000000093|3388791.1500000004|
|Clerk#000000124|        1683034.58|
+---------------+------------------+
only showing top 5 rows



In [19]:
ordersDf.printSchema()

root
 |-- O_ORDERKEY: integer (nullable = true)
 |-- O_CUSTKEY: integer (nullable = true)
 |-- O_ORDERSTATUS: string (nullable = true)
 |-- O_TOTALPRICE: double (nullable = true)
 |-- O_ORDERDATE: date (nullable = true)
 |-- O_ORDERPRIORITY: string (nullable = true)
 |-- O_CLERK: string (nullable = true)
 |-- O_SHIPPRIORITY: integer (nullable = true)
 |-- O_COMMENT: string (nullable = true)

